# Introducción

Todo sistema en algún grado inteligente depende absolutamente de la realimentación que recibe de su entorno. Es decir, el funcionamiento correcto o incorrecto de los sensores que posea es crucial para su eficaz funcionamiento. Luego, es preciso calibrar aquellos sensores para conocer su comportamiento bajo distintas condiciones ambientales, y así obrar adecuadamente a la hora de diseñar el sistema que realizamos. Desafortunadamente, en el caso de los sensores ópticos (cámaras) ocurre que es complejo determinar de forma directa las variables de interés en la calibración (no son usualmente suministradas por el fabricante, adicionalmente, medirlas experimentalmente no es una tarea fácil). Sin embargo, usando modelos geométricos y matemáticos existentes, es posible obtener los parámetros de calibración de una cámara de forma indirecta. Este documento pretende ilustrar el modelo más sencillo que se concibe de una cámara, y luego demostrar el proceso de calbración usando la librería [OpenCV](http://opencv.org) (cuyas funciones se basan en el modelo mencionado).

## El modelo *pinhole* <cite data-cite="bradski2008learning"></cite>

Este modelo de cámara consiste en un plano con una apertura puntual en su centro, sobre la que sólo puede pasar un rayo de luz. De esta manera, sólo un rayo que parta de algún punto específico del espacio puede entrar. Paralelo a este plano, existe otro plano, que es el que capturará los rayos proyectados, formándose así una imagen.

![Modelo pinhole: los rayos proyectados a través de la apertura forman una imagen.](img/1-fig1.png "Modelo pinhole")

La distancia $f$ es conocida como *distancia focal* y $Z$ es la distancia entre cámara y objeto. Para simplificar los cálculos necesarios en la solución de el sistema coordenado de la anterior imagen, se reorganiza todo así:

![Modelo usado para la solución algebraica. Nótese la diferenciación entre coordenadas $x$,$y$ (imagen) y $X$,$Y$,$Z$ (espacio)](img/1-fig2.png "Modelo pinhole")

El mapeo $f:\ Q \mapsto q$ corresponde a la siguiente función:

$$f\left(X,Y,Z\right) = \left(f_x\left(\frac{X}{Z}\right)+c_x,\ f_y\left(\frac{Y}{Z}\right)+c_y\right)$$

En donde,
\begin{equation} \label{eq:mapx}
    x_{imagen} = f_x\left(\frac{X}{Z}\right)+c_x
\end{equation}
\begin{equation} \label{eq:mapy}
    y_{imagen} = f_y\left(\frac{Y}{Z}\right)+c_y
\end{equation}

La figura ilustra lo que sería un modelo ideal, en que el plano de la imagen es paralelo al plano $\hat{i}\hat{j}$, y el eje $\hat{k}$ interseca al plano de la imagen en su centro. En la construcción de una cámara, lograr una alineación perfecta es imposible. Por ello, se introducen las variables $c_x$ y $c_y$, las cuales representan desplazamientos respectivamente horizontal y vertical del eje óptico respecto al centro del plano. También se introducen dos distancias focales, $f_x$ y $f_y$, debido a que los sensores ópticos no poseen pixeles cuadrados, en realidad son rectangulares.

Reorganizando las ecuaciones \ref{eq:mapx} y \ref{eq:mapy},

$$x_{imagen} Z = f_x X + c_x Z$$
$$y_{imagen} Z = f_y Y + c_y Z$$

Es conveniente expresar estas ecuaciones de forma matricial usando coordenadas homogéneas, de la forma $q = MQ$, en donde

\begin{center}
$q =
\begin{bmatrix}
    x \\[0.3em]
    y \\[0.3em]
    w
\end{bmatrix}$ , 
$M = 
\begin{bmatrix}
f_x &  0  & c_x  \\[0.3em]
 0  & f_y & c_y  \\[0.3em]
 0  &  0  &  1
\end{bmatrix}$ ,
$Q =
\begin{bmatrix}
    X \\[0.3em]
    Y \\[0.3em]
    Z
\end{bmatrix}$
\end{center}

### Distorsión de los lentes

Varios supuestos deben olvidarse, y reemplazarse por modelos, para poder llegar a una representación más precisa y generalizada. No se consideró previamente, pero los sensores ópticos de una cámara (CCDs, o dispositivos de carga acoplada, son los más ampliamente usados) poseen una alta integración en una área muy reducida, 